# Simulation Speed Comparison

Use this notebook to compare the simulation execution times of a
communication channel Nengo model (both learned and non-learning variants)
between the FPGA board and on your PC. The Nengo model size will be varied
by the number of neurons, and a plot of the simulation wall times will be
generated.

## Step 1: Set up the Python Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

import nengo
from nengo.processes import WhiteSignal
from nengo.solvers import NoSolver

import nengo_fpga
from nengo_fpga.networks import FpgaPesEnsembleNetwork

## Step 2: Choose an FPGA Device

Define the FPGA device on which the remote FpgaPesEnsembleNetwork will
run. This name corresponds with the name in your `fpga_config` file.
Recall that in the `fpga_config` file, device names are identified by the
square brackets (e.g., **[de1]** or **[pynq]**). The names defined in
your configuration file might differ from the example below. Here, the
device **de1** is being used.

In [ ]:
board = 'de1'  # Change this to your desired device name

## Step 3: Define Model Creation Functions

Since the simulation timing code creates the same Nengo model for each
simulation run (the only difference between each run being the number of
neurons in the neural ensemble), the following model creation functions
are defined to reduce code duplication.

Note that in the code below, the `nengo.solver.NoSolver` connection solver
is being used to bypass the regular connection weights solver. This is to
avoid the potentially long model build times for large ensemble sizes.

### Step 3a: Model creation functions for native Nengo

In [ ]:
# Model creation function for a non-learned communication channel
# using the native (python) Nengo simulator
def make_native_model(n_neurons, neuron_type, dimensions=1):
    # Create a nengo Network that can be returned by this function
    model = nengo.Network()

    with model:
        # Input white noise signal
        input_node = nengo.Node(WhiteSignal(60, high=5),
                                size_out=dimensions)
        # Output node
        output_node = nengo.Node(size_in=dimensions)

        # Neural ensemble
        ens = nengo.Ensemble(n_neurons, dimensions=dimensions,
                             neuron_type=neuron_type)

        # Make the input and output connections
        nengo.Connection(input_node, ens)
        nengo.Connection(
            ens, output_node,
            solver=NoSolver(np.random.random((n_neurons, dimensions)))
        )

    return model

# Model creation function for a learned communication channel
# using the native (python) Nengo simulator
def make_native_learned_model(n_neurons, neuron_type, dimensions=1):
    # Create a nengo Network that can be returned by this function
    model = nengo.Network()

    with model:
        # White-noise reference signal
        input_node = nengo.Node(WhiteSignal(60, high=5),
                                size_out=dimensions)
        # Output node
        output_node = nengo.Node(size_in=dimensions)

        # Neural ensemble
        ens = nengo.Ensemble(n_neurons, dimensions=dimensions,
                             neuron_type=neuron_type)

        # Make the input and output connections
        nengo.Connection(input_node, ens)
        conn = nengo.Connection(
            ens, output_node,
            solver=NoSolver(np.random.random((n_neurons, dimensions)))
        )
        # Set the learning rule on the output connection
        conn.learning_rule_type = nengo.PES(learning_rate=5e-5)

        # Compute the learned error signal and connect it to the
        # learning rule of the output connection
        error = nengo.Node(size_in=dimensions)
        nengo.Connection(output_node, error)
        nengo.Connection(input_node, error, transform=-1)
        nengo.Connection(error, conn.learning_rule)

    return model

### Step 3b: Model creation functions for NengoFPGA models

In [ ]:
# Model creation function for a non-learned communication channel
# using the NengoFPGA simulator
def make_fpga_model(n_neurons, neuron_type, board, dimensions=1):
    # Create a nengo Network that can be returned by this function
    model = nengo.Network()

    with model:
        # White-noise reference signal
        input_node = nengo.Node(WhiteSignal(60, high=5),
                                size_out=dimensions)

        # Remote FPGA neural ensemble
        fpga_ens = FpgaPesEnsembleNetwork(
            board, n_neurons=n_neurons, dimensions=dimensions,
            learning_rate=0
        )

        # Set the FPGA ensemble neuron type and connection solver
        fpga_ens.ensemble.neuron_type = neuron_type
        fpga_ens.connection.solver = NoSolver(
            np.random.random((n_neurons, dimensions)))

        # Connect the input to the FPGA ensemble
        nengo.Connection(input_node, fpga_ens.input)

    return model

# Model creation function for a non-learned communication channel
# using the NengoFPGA simulator
def make_fpga_learned_model(n_neurons, neuron_type, board, dimensions=1):
    # Create a nengo Network that can be returned by this function
    model = nengo.Network()

    with model:
        # White-noise reference signal
        input_node = nengo.Node(WhiteSignal(60, high=5),
                                size_out=dimensions)

        # Remote FPGA neural ensemble
        fpga_ens = FpgaPesEnsembleNetwork(
            board, n_neurons=n_neurons, dimensions=dimensions,
            learning_rate=5e-5
        )

        # Set the FPGA ensemble neuron type and connection solver
        fpga_ens.ensemble.neuron_type = neuron_type
        fpga_ens.connection.solver = NoSolver(
            np.random.random((n_neurons, dimensions)))

        # Connect the input to the FPGA ensemble
        nengo.Connection(input_node, fpga_ens.input)

        # Create a Nengo node to calculate the error signal
        error = nengo.Node(size_in=dimensions)

        # Compute the error (error = actual - target) using transforms
        # on the connections to the error ensemble
        nengo.Connection(fpga_ens.output, error)
        nengo.Connection(input_node, error, transform=-1)

        # Project error to the adaptive neural ensemble on the FPGA
        nengo.Connection(error, fpga_ens.error)

    return model

## Step 4: Time the Simulation Runs

Using the model creation functions above, the time it takes to run each
simulation can be computed. The process of creating and timing each
simulation is as follows:

1. Create the Nengo model using the appropriate model creation function from above. Pass the function the desired model parameters (number of neurons, etc.)
1. Create the Nengo simulator object using the model created in the previous step. Since this step is independent of the simulation run, do not time this step.
1. Start the simulation wall timer.
1. Run the simulation for 10 seconds.
1. Stop the simulation wall timer, and compute the elapsed simulation wall time.

Note that between each simulation run on the FPGA, the script is paused
for 30 seconds to allow the FPGA resources to be completely freed.

In [ ]:
# Uncomment the line below to enable SSH logging information to be
# printed to the Jupyter notebook output.
# nengo.utils.logging.log('info')

# Create lists to store the simulation wall times for each type
# of model
times_native = []
times_native_learned = []
times_fpga = []
times_fpga_learned = []

# The number of neurons to run the simulation for
n_neuron_list = [100, 500, 1000, 5000, 10000, 15000]

# The neuron type and number of dimensions used for all of the simulation
# runs.
neuron_type = nengo.neurons.RectifiedLinear()
dimensions = 1

# Run the simulations for each number of neurons
for n_neurons in n_neuron_list:
    # Print the current number of neurons being processed
    print("Processing n_neurons=%i" % n_neurons)

    # Create the various Nengo models and respective Nengo simulators
    model_native = make_native_model(
        n_neurons, neuron_type, dimensions)
    sim_native = nengo.Simulator(model_native)

    model_native_learned = make_native_learned_model(
        n_neurons, neuron_type, dimensions)
    sim_native_learned = nengo.Simulator(model_native_learned)

    model_fpga = make_fpga_model(
        n_neurons, neuron_type, board, dimensions)
    sim_fpga = nengo_fpga.Simulator(model_fpga)

    model_fpga_learned = make_fpga_learned_model(
        n_neurons, neuron_type, board, dimensions)
    sim_fpga_learned = nengo_fpga.Simulator(model_fpga_learned)

    # Time each simulation run, and append the results to the
    # respective results list
    t_start = time.time()
    sim_native.run(10)
    t_end = time.time()
    times_native.append(t_end - t_start)
    print("Done native run", end=", ")

    t_start = time.time()
    sim_native_learned.run(10)
    t_end = time.time()
    times_native_learned.append(t_end - t_start)
    print("Done native-lrn run", end=", ")

    t_start = time.time()
    sim_fpga.run(10)
    t_end = time.time()
    times_fpga.append(t_end - t_start)
    time.sleep(30)
    print("Done fpga run", end=", ")

    t_start = time.time()
    sim_fpga_learned.run(10)
    t_end = time.time()
    times_fpga_learned.append(t_end - t_start)
    time.sleep(30)
    print("Done fpga-lrn run", end=", ")

    print("\n")

# Print a message indicating all runs have been completed
print("Done all runs!")

## Step 5: Plot the Results

In [ ]:
# Plot figure
plt.figure(figsize=(16, 8))
plt.plot(n_neuron_list, times_native, '-x', label='Native')
plt.plot(n_neuron_list, times_native_learned, '-x', label='Native Learned')
plt.plot(n_neuron_list, times_fpga, '-x', label='FPGA')
plt.plot(n_neuron_list, times_fpga_learned, '-x', label='FPGA Learned')

plt.legend(loc='upper left')
plt.xlabel("Number of Neurons")
plt.ylabel("Wall time (s)")
plt.title("Simulation Wall Times (Native vs FPGA)");